# Observations

- No missing values (good)
- There are features with slight colleration w.r.t. the Attrition (they are also explainable)
- There are three features immediately to be dropped because of 1 distinct value
- We are left with 19 categorical and 13 numerical features
- ID type of feature is also excluded (numerical features down to 12)
- Number of numerical features gone up to 23, nominals down to 7, because of instructions

In [2]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import LabelEncoder

from sklearn.neighbors import KNeighborsClassifier as KNC
from sklearn.ensemble import RandomForestClassifier as RFC
from sklearn.ensemble import GradientBoostingClassifier as GBC

import sweetviz as sv

from utils.evaluation import *

In [7]:
df = pd.read_csv("data/use_case_employee-attrition.csv")
df = df.drop(["EmployeeCount", "Over18", "StandardHours", "EmployeeNumber"], axis=1)

nominals = ["BusinessTravel", "Department", "EducationField", "Gender", "JobRole", "MaritalStatus", "OverTime"]
numericals = ["Age", "DailyRate", "DistanceFromHome", "Education", "EnvironmentSatisfaction", "HourlyRate", "JobInvolvement", "JobLevel", "JobSatisfaction", "MontlyIncome", "MonthlyRate", "NumCompaniesWorked",
              "PercentSalaryHike", "PerformanceRating", "RelationshipSatisfaction", "StockOptionLevel", "TotalWorkingYears", "TrainingTimesLastYear", "WorkLifeBalance", "YearsAtCompany", "YearsInCurrentRole", 
              "YearsSinceLastPromotion", "YearsWithCurrManager"]

df_n = df.drop(nominals, axis=1)

X = df_n.drop("Attrition", axis=1)
y = df_n["Attrition"]
y = LabelEncoder().fit_transform(y)

model = KNC(n_neighbors=1)

compute_acc_rec_prec_f1_with_cv(model, X, y, cv_n=10)

Accuracy score: 75.98639455782312 +/- 2.2572326561955163
Recall score: 25.32608695652174 +/- 9.445270668234938
Precision score: 25.044475190909825 +/- 6.615462101207091
F1 score: 24.964967522706306 +/- 7.644837154110569


({'accuracy': 75.98639455782312,
  'recall': 25.32608695652174,
  'precision': 25.044475190909825,
  'f1': 24.964967522706306},
 {'accuracy': 2.2572326561955163,
  'recall': 9.445270668234938,
  'precision': 6.615462101207091,
  'f1': 7.644837154110569})

In [14]:
report = sv.analyze(df)
report.show_html("report.html", open_browser=False)

                                             |          | [  0%]   00:00 -> (? left)

Report report.html was generated.
